In [ ]:
!pip install transformers

In [ ]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time

2. Mengambil Dataset

In [ ]:
#Memasukkan path data train
path = Path('/content/train_agro_dong (1).json')

#Membuka file .json
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []
queries = []
answers = []

#Pencocokan konteks, pertanyaan, dan jawaban
for group in squad_dict['data']:
    for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            for answer in qa['answers']:
#Menyimpan konteks, pertanyaan, dan jawaban ke dalam list
                texts.append(context)
                queries.append(question)
                answers.append(answer)

train_texts, train_queries, train_answers = texts, queries, answers

In [ ]:
#Memasukkan path data validation
path = Path('/content/val_agro_dong.json')

#Membuka file .json
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []
queries = []
answers = []

#Pencocokan konteks, pertanyaan, dan jawaban
for group in squad_dict['data']:
    for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            for answer in qa['answers']:
#Menyimpan konteks, pertanyaan, dan jawaban ke dalam list
                texts.append(context)
                queries.append(question)
                answers.append(answer)

val_texts, val_queries, val_answers = texts, queries, answers

3. Cek Data

In [ ]:
#Jumlah Data Train
print(len(train_texts))
print(len(train_queries))
print(len(train_answers))

99
99
99


In [ ]:
a = 1

print("Konteks: ",train_texts[a])
print("Pertanyaan: ",train_queries[a])
print("Jawaban: ",train_answers[a])

#ket:
#1. huruf besar dan kecil mempengaruhi nilai answer_start
#2. jawaban harus mengutip isi dari konteks
#3. tidak boleh menghilangkan imbuhan
#4. tidak boleh menambahkan satu kata ataupun simbol
#5. tidak boleh menggunakan tanda kutip dua (" ")
#6. kelebihan atau kekurangan spasi akan mempengaruhi nilai answer_start

Konteks:  Hama dan penyakit tumbuhan merupakan jenis organisme pengganggu tumbuhan (OPT), selain gulma. Serangan hama dan penyakit pada tanaman dapat menyebabkan kerugian besar pada tanaman dan dapat mengancam perekonomian petani. Penyebaran hama dan penyakit tanaman meningkat drastis dalam beberapa tahun terakhir. Hama dan penyakit tanaman mudah menyebar ke beberapa negara dan mencapai proporsi epidemi. Belalang, lalat buah, ulat grayak, penyakit antaknose, fuso, penyakit virus kerdil, busuk buah adalah beberapa hama dan penyakit tanaman yang paling merusak. Tiga cara penyebaran hama dan penyakit tanaman yaitu dengan cara: 1) perdagangan atau migrasi 2) pengaruh lingkungan, seperti faktor cuaca, angin, percikan air hujan, dan 3) faktor biotik berupa: serangga atau vektor lainnya.
Pertanyaan:  Apa saja beberapa hama dan penyakit tanaman yang merusak?
Jawaban:  {'answer_start': 227, 'text': 'Belalang, lalat buah, ulat grayak, penyakit antaknose, fuso, penyakit virus kerdil, busuk buah a

In [ ]:
#Jumlah Data Validation
print(len(val_texts))
print(len(val_queries))
print(len(val_answers))

9
9
9


In [ ]:
b = 2

print("Konteks: ",val_texts[b])
print("Pertanyaan: ",val_queries[b])
print("Jawaban: ",val_answers[b])

Konteks:  Penularan virus dari tanaman sakit ke tanaman sehat dapat terjadi secara mekanik yaitu melalui luka, penyambungan (grafting), vektor serangga misalnya serangga contohnya kutu daun, nematoda, tungau dan cendawan, penularan juga dapat terjadi melalui bahan perbanyakan vegetatif, benih dan polen. Misalnya Tomato mosaic virus dapat ditularkan melalui biji dan kontak langsung. Untuk mendeteksi adanya infeksi virus disuatu tanaman maka kita harus mengenali adanya gejala virus. Selain untuk mendeteksi virus, gejala tanaman yang terinfeksi virus ditujukan juga untuk pemberian nama virus dan menunjukkan adanya infeksi virus tanaman.
Pertanyaan:  Bagaimana virus dapat menular dari tanaman sakit ke tanaman sehat?
Jawaban:  {'answer_start': 0, 'text': 'Penularan virus dari tanaman sakit ke tanaman sehat dapat terjadi secara mekanik yaitu melalui luka, penyambungan (grafting), vektor serangga misalnya serangga contohnya kutu daun, nematoda, tungau dan cendawan, penularan juga dapat terjad

4. Penentuan end index

In [ ]:
cek = "Permasalahan organisme pengganggu tumbuhan di lapangan seperti serangga dan penyakit dapat dicegah dengan menjaga kesehatan tanaman yaitu budidaya tanaman sehat, misalnya dengan menggunakan benih bersertifikat, memilih varietas tanaman yang tahan penyakit, melakukan rotasi tanaman yang bukan satu famili dengan tanaman yang ditanam di area yang sama selama beberapa tahun berturut-turut, mempertahankan nutrisi tanah seperti pemberian pupuk dengan dosis yang tepat. Pengolahan tanaman juga dapat dilakukan dengan cara irigasi yaitu mengairi tanaman dengan benar. Menjaga agar pertanaman bebas dari gulma, karena gulma dapat berpotensi menjadi sumber inokulum bagi patogen dan merupakan inang alternatif bagi hama dan penyakit yang menyerang pertanaman. Penggunaan pestisida kimia merupakan alternatif terakhir, karena penggunaan pestisida yang terlalu sering akan berdampak negatif bagi lingkungan dan dapat membunuh serangga yang menguntungkan."
cek[398]

'a'

In [ ]:
for indeks, karakter in enumerate(cek):
    print(f"Karakter: {karakter}, Indeks: {indeks}")


Karakter: P, Indeks: 0
Karakter: e, Indeks: 1
Karakter: r, Indeks: 2
Karakter: m, Indeks: 3
Karakter: a, Indeks: 4
Karakter: s, Indeks: 5
Karakter: a, Indeks: 6
Karakter: l, Indeks: 7
Karakter: a, Indeks: 8
Karakter: h, Indeks: 9
Karakter: a, Indeks: 10
Karakter: n, Indeks: 11
Karakter:  , Indeks: 12
Karakter: o, Indeks: 13
Karakter: r, Indeks: 14
Karakter: g, Indeks: 15
Karakter: a, Indeks: 16
Karakter: n, Indeks: 17
Karakter: i, Indeks: 18
Karakter: s, Indeks: 19
Karakter: m, Indeks: 20
Karakter: e, Indeks: 21
Karakter:  , Indeks: 22
Karakter: p, Indeks: 23
Karakter: e, Indeks: 24
Karakter: n, Indeks: 25
Karakter: g, Indeks: 26
Karakter: g, Indeks: 27
Karakter: a, Indeks: 28
Karakter: n, Indeks: 29
Karakter: g, Indeks: 30
Karakter: g, Indeks: 31
Karakter: u, Indeks: 32
Karakter:  , Indeks: 33
Karakter: t, Indeks: 34
Karakter: u, Indeks: 35
Karakter: m, Indeks: 36
Karakter: b, Indeks: 37
Karakter: u, Indeks: 38
Karakter: h, Indeks: 39
Karakter: a, Indeks: 40
Karakter: n, Indeks: 41
Ka

In [ ]:
indeks_hama = cek.find("gulma")
print("Indeks kata :", indeks_hama)


Indeks kata : 599


In [ ]:
for answer, text in zip(train_answers, train_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    #Nilai end index sebenarnya
    end_idx = start_idx + len(real_answer)

    # print(start_idx)
    # print(end_idx)
    # print(end_idx-start_idx)
    # print(len(real_answer))
    # print(text[start_idx:end_idx],'\n')
    #print(text[start_idx:end_idx] == real_answer)
    #Ketika real answer sesuai
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    #Ketika real answer lebih dari 1 karakter
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    #Ketika real answer lebih dari 2 karakter
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2
    else:
        answer['answer_end'] = end_idx + 1

In [ ]:
train_answers

In [ ]:
for answer, text in zip(val_answers, val_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    #Nilai end index sebenarnya
    end_idx = start_idx + len(real_answer)

    #Ketika real answer sesuai
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    #Ketika real answer lebih dari 1 karakter
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    #Ketika real answer lebih dari 2 karakter
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2
    else:
        answer['answer_end'] = end_idx + 1

In [ ]:
for answer, text in zip(val_answers, val_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(real_answer)

    # Ketika real answer sesuai
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # Ketika real answer lebih dari 1 karakter
    elif text[start_idx - 1:end_idx - 1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # Ketika real answer lebih dari 2 karakter
    elif text[start_idx - 2:end_idx - 2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2
    # Tambahkan kondisi untuk real answer lebih dari 3 karakter
    elif text[start_idx - 3:end_idx - 3] == real_answer:
        answer['answer_start'] = start_idx - 3
        answer['answer_end'] = end_idx - 3
    # Tambahkan kondisi untuk real answer lebih dari 4 karakter
    elif text[start_idx - 4:end_idx - 4] == real_answer:
        answer['answer_start'] = start_idx - 4
        answer['answer_end'] = end_idx - 4
    # Tambahkan kondisi untuk real answer lebih dari 5 karakter
    elif text[start_idx - 5:end_idx - 5] == real_answer:
        answer['answer_start'] = start_idx - 5
        answer['answer_end'] = end_idx - 5
    # Tambahkan kondisi untuk real answer lebih dari 6 karakter
    elif text[start_idx - 6:end_idx - 6] == real_answer:
        answer['answer_start'] = start_idx - 6
        answer['answer_end'] = end_idx - 6
    # Tambahkan kondisi untuk real answer lebih dari 7 karakter
    elif text[start_idx - 7:end_idx - 7] == real_answer:
        answer['answer_start'] = start_idx - 7
        answer['answer_end'] = end_idx - 7
    # Tambahkan kondisi untuk real answer lebih dari 8 karakter
    elif text[start_idx - 8:end_idx - 8] == real_answer:
        answer['answer_start'] = start_idx - 8
        answer['answer_end'] = end_idx - 8
    # Tambahkan kondisi untuk real answer lebih dari 9 karakter
    elif text[start_idx - 9:end_idx - 9] == real_answer:
        answer['answer_start'] = start_idx - 9
        answer['answer_end'] = end_idx - 9
    # Dan seterusnya, sesuai dengan jumlah karakter yang Anda ingin tangani
    # Anda dapat melanjutkan pola ini sesuai kebutuhan


5. Tokenisasi Passages dan Queries

In [ ]:
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("Rifky/Indobert-QA")

train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


6. Convert The Start-End Positions to Tokens Start-End Positions

In [ ]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []

  count = 0

  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length

    # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
    if end_positions[-1] is None:
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - 1)
      # if end position is still None the answer passage has been truncated
      if end_positions[-1] is None:
        count += 1
        end_positions[-1] = tokenizer.model_max_length

  print(count)

  #Update data
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

2
0


In [ ]:
train_encodings

In [ ]:
# def add_token_positions(encodings, answers):
#     start_positions = []
#     end_positions = []

#     count = 0

#     for i in range(len(answers)):
#         answer_start = answers[i].get('answer_start')
#         answer_end = answers[i].get('answer_end')

#         if answer_start is not None and answer_end is not None:
#             start_positions.append(encodings.char_to_token(i, answer_start))
#             end_positions.append(encodings.char_to_token(i, answer_end))

#             # jika posisi awal None, artinya potongan jawaban telah dipotong
#             if start_positions[-1] is None:
#                 start_positions[-1] = tokenizer.model_max_length

#             # jika posisi akhir None, fungsi 'char_to_token' menunjuk ke spasi setelah token yang benar, maka tambahkan - 1
#             if end_positions[-1] is None:
#                 end_positions[-1] = encodings.char_to_token(i, answer_end - 1)
#                 # jika posisi akhir masih None, artinya potongan jawaban telah dipotong
#                 if end_positions[-1] is None:
#                     count += 1
#                     end_positions[-1] = tokenizer.model_max_length

#     print(count)

#     # Perbarui data
#     encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# add_token_positions(train_encodings, train_answers)
# add_token_positions(val_encodings, val_answers)


2
0


7. Membuat Kelas Dataset

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
import torch
from torch.utils.data import Dataset

class SquadDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# Contoh penggunaan
# Anda perlu menggantinya dengan data sebenarnya yang Anda miliki
# Misalnya, encodings = {'input_ids': [...], 'attention_mask': [...], ...}
# dataset = SquadDataset(encodings)


In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
#Menggunakan Dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
#Menggunakan GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Working on:', device)

Working on: cpu


8. Membangun Model

In [ ]:
model = BertForQuestionAnswering.from_pretrained("indolem/indobert-base-uncased").to(device)

#optim = AdamW(model.parameters(), lr=5e-5)
#optim = AdamW(model.parameters(), lr=4e-5)
#optim = AdamW(model.parameters(), lr=3e-5)
#optim = AdamW(model.parameters(), lr=2e-5)
optim = AdamW(model.parameters(), lr=1e-5)

epochs = 5

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


9. Train and Evaluate Model

In [ ]:
whole_train_eval_time = time.time()

train_losses = []
val_losses = []

print_every = 10

for epoch in range(epochs):
  epoch_time = time.time()

  #train mode
  model.train()

  loss_of_epoch = 0

  print("############Train############")

  for batch_idx,batch in enumerate(train_loader):

    optim.zero_grad()

    your_max_value = 9223372036854775807

    input_ids = batch['input_ids'].clamp(max=your_max_value).to(device)
    attention_mask = batch['attention_mask'].clamp(max=your_max_value).to(device)
    start_positions = batch['start_positions'].clamp(max=your_max_value).to(device)
    end_positions = batch['end_positions'].clamp(max=your_max_value).to(device)


    # input_ids = batch['input_ids'].to(device)
    # attention_mask = batch['attention_mask'].to(device)
    # start_positions = batch['start_positions'].to(device)
    # end_positions = batch['end_positions'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    #backwards pass
    loss.backward()
    #update bobot
    optim.step()
    #Hitung nilai loss
    loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
      print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(train_loader)
  train_losses.append(loss_of_epoch)

  ##########Evaluation##################

  #evaluation mode
  model.eval()

  print("############Evaluate############")

  loss_of_epoch = 0

  for batch_idx,batch in enumerate(val_loader):

    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss = outputs[0]
      #Hitung nilai loss
      loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
       print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(val_loader)
  val_losses.append(loss_of_epoch)

  #Print hasil tiap epoch
  print("\n-------Epoch ", epoch+1,
        "-------"
        "\nTraining Loss:", train_losses[-1],
        "\nValidation Loss:", val_losses[-1],
        "\nTime: ",(time.time() - epoch_time),
        "\n-----------------------",
        "\n\n")

print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))

############Train############


RuntimeError: ignored

In [ ]:
#Save model

torch.save(model,"/content/drive/MyDrive/Colab Notebooks/BERT/Model_Terjemahan_Arbain_v.3/Panjang_v.3_e(5),l(0,00001),b(8)")

10. Plot Train & Val Loss

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(15,10))

ax.set_title("Train and Validation Losses",size=20)
ax.set_ylabel('Loss', fontsize = 20)
ax.set_xlabel('Epochs', fontsize = 25)
_=ax.plot(train_losses)
_=ax.plot(val_losses)
_=ax.legend(('Train','Val'),loc='upper right')